# 🧮 Building Analysis: Computational Tools for Compliance

This notebook demonstrates **practical computational tools** that our compliance agent uses to analyze buildings.

**Goal**: Show you the tools the agent has available for sophisticated building analysis.

**Building**: Real Vilamalla Industrial Complex (9 levels, 23 doors, 102 walls)


## 🚀 Step 1: Load Real Building Data

Let's load the Vilamalla building data that our agent works with:

In [1]:
# Load building data and computational tools
import sys
sys.path.append('..')

from scripts.load_building_data import load_vilamalla_building
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from pathlib import Path

# Load the enhanced building data
print("🏗️ Loading Vilamalla Industrial Complex...")
loader = load_vilamalla_building()

print(f"✅ Building Data Loaded:")
print(f"   Project: {loader.metadata.get('project_name')}")
print(f"   Levels: {len(loader.levels)}")
print(f"   Rooms: {len(loader.all_rooms)}")
print(f"   Doors: {len(loader.all_doors)}")
print(f"   Walls: {len(loader.all_walls)}")
print(f"   Total area: {loader.metadata.get('total_area', 0):.0f} m²")

INFO:scripts.load_building_data:Loaded building data: 2111B - 9 levels, 9 rooms, 23 doors, 102 walls
INFO:scripts.load_building_data:Enhanced 23 door connections


🏗️ Loading Vilamalla Industrial Complex...
✅ Building Data Loaded:
   Project: 2111B
   Levels: 9
   Rooms: 9
   Doors: 23
   Walls: 102
   Total area: 720 m²


## 🔗 Step 2: Building Connectivity Analysis

Let's analyze how rooms are connected and find critical circulation points:

In [ ]:
# Analyze building connectivity with VISUALIZATION
from src.calculations.graph import create_circulation_graph, find_critical_circulation_points
from src.schemas import Project
import json
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Convert loader data to Project schema for analysis
def create_project_from_loader(loader):
    """Convert loader data to Project schema format."""
    project_data = {
        "metadata": loader.metadata,
        "levels": loader.levels
    }
    return Project(**project_data)

# Create project object
project = create_project_from_loader(loader)

print("🔗 Building Connectivity Analysis:")
print("=" * 60)

# Create circulation graph
graph = create_circulation_graph(project)
stats = graph.get_graph_statistics()

print(f"📊 Graph Statistics:")
print(f"   Nodes (rooms): {stats['total_nodes']}")
print(f"   Edges (doors): {stats['total_edges']}")
print(f"   Connected: {'✅ Yes' if stats['is_connected'] else '❌ No'}")
print(f"   Connected Components: {stats['connected_components']}")
print(f"   Average Connections per Room: {stats['average_degree']:.1f}")

# Find critical circulation points
critical_points = find_critical_circulation_points(project)

print(f"\n🎯 Critical Circulation Analysis:")
print(f"   Critical Rooms: {critical_points['critical_room_count']}")
print(f"   Connectivity Risk: {critical_points['connectivity_risk'].upper()}")

# 🎨 VISUALIZATION: Circulation Graph
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Left: Network graph visualization
G = graph.graph
if G.number_of_nodes() > 0:
    # Use spring layout for better visualization
    pos = nx.spring_layout(G, k=3, iterations=50)
    
    # Draw nodes (rooms)
    node_colors = []
    node_sizes = []
    
    for node in G.nodes():
        # Color critical rooms differently
        is_critical = any(room['room_id'] == node.replace('room_', '') 
                         for room in critical_points.get('critical_rooms', []))
        node_colors.append('red' if is_critical else 'lightblue')
        
        # Size by connectivity
        degree = G.degree(node)
        node_sizes.append(300 + degree * 200)
    
    # Draw the graph
    nx.draw(G, pos, ax=ax1, 
           node_color=node_colors, 
           node_size=node_sizes,
           with_labels=True,
           labels={node: node.replace('room_', '').replace('_DEFAULT', '') for node in G.nodes()},
           font_size=7,
           font_weight='bold',
           edge_color='gray',
           width=2,
           alpha=0.8)
    
    ax1.set_title('Building Circulation Graph\n(Red = Critical Rooms)', fontsize=14, fontweight='bold')
    ax1.text(0.02, 0.98, 'Node size = connectivity\nRed = critical for circulation', 
            transform=ax1.transAxes, va='top', ha='left', 
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
else:
    ax1.text(0.5, 0.5, 'No graph connections\navailable', 
            ha='center', va='center', transform=ax1.transAxes, fontsize=12)
    ax1.set_title('Circulation Graph', fontsize=14)

# Right: Connectivity statistics chart
room_connections = {}
for room in loader.all_rooms:
    room_id = room['id']
    room_node = f"room_{room_id}"
    if room_node in G:
        connections = G.degree(room_node)
        room_connections[room_id] = connections
    else:
        room_connections[room_id] = 0

if room_connections:
    rooms = list(room_connections.keys())
    connections = list(room_connections.values())
    
    # Create horizontal bar chart
    colors = ['red' if c == 0 else 'orange' if c == 1 else 'lightgreen' for c in connections]
    bars = ax2.barh(range(len(rooms)), connections, color=colors, alpha=0.7, edgecolor='black')
    
    ax2.set_yticks(range(len(rooms)))
    ax2.set_yticklabels([r.split('_')[1] if '_' in r else r for r in rooms], fontsize=8)
    ax2.set_xlabel('Number of Connections', fontweight='bold')
    ax2.set_title('Room Connectivity Levels', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='x')
    
    # Add value labels on bars
    for i, (bar, value) in enumerate(zip(bars, connections)):
        ax2.text(value + 0.05, i, str(value), va='center', fontweight='bold')
    
    # Add color legend
    ax2.text(0.02, 0.98, 'Red = Isolated\nOrange = Poor\nGreen = Well Connected', 
            transform=ax2.transAxes, va='top', ha='left',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

print(f"\n🎨 Visualization shows:")
print(f"   📊 Left: Network graph with critical rooms in red")
print(f"   📈 Right: Room connectivity levels (green = well connected)")

## 🚨 Step 3: Evacuation Route Analysis

Now let's analyze evacuation routes for all rooms:

In [3]:
# Comprehensive evacuation analysis
from src.calculations.graph import find_all_evacuation_routes, validate_evacuation_compliance

print("🚨 Evacuation Route Analysis:")
print("=" * 60)

# Find all evacuation routes
all_routes = find_all_evacuation_routes(project)

print(f"📊 Evacuation Statistics:")
print(f"   Total Rooms: {all_routes['total_rooms']}")
print(f"   Average Distance: {all_routes['average_distance']:.1f} m")
print(f"   Compliance Rate: {all_routes['compliance_summary']['compliance_rate']:.1%}")

if all_routes['longest_route']:
    longest = all_routes['longest_route']
    print(f"\n🔍 Longest Evacuation Route:")
    print(f"   Room: {longest['room_id']}")
    print(f"   Distance: {longest['distance']:.1f} m")
    print(f"   Path: {' → '.join(longest['path'])}")
    print(f"   Accessible: {'✅ Yes' if longest['is_accessible'] else '❌ No'}")

if all_routes['shortest_route']:
    shortest = all_routes['shortest_route']
    print(f"\n⚡ Shortest Evacuation Route:")
    print(f"   Room: {shortest['room_id']}")
    print(f"   Distance: {shortest['distance']:.1f} m")

# Validate compliance
compliance = validate_evacuation_compliance(project, max_distance=25.0)

print(f"\n📋 Compliance Validation:")
print(f"   Overall Status: {compliance['overall_status']}")
print(f"   Compliant Rooms: {len(compliance['compliant_rooms'])}/{compliance['total_rooms']}")
print(f"   Non-Compliant: {len(compliance['non_compliant_rooms'])}")
print(f"   Inaccessible: {len(compliance['inaccessible_rooms'])}")

if compliance['non_compliant_rooms']:
    print(f"\n❌ Non-Compliant Rooms (exceed {compliance['max_distance_limit']}m):")
    for room in compliance['non_compliant_rooms'][:5]:  # Show first 5
        print(f"   🏠 {room['room_id']}: {room['distance']:.1f}m (excess: {room['excess_distance']:.1f}m)")

if compliance['inaccessible_rooms']:
    print(f"\n🚫 Inaccessible Rooms (no evacuation path):")
    for room in compliance['inaccessible_rooms']:
        print(f"   🏠 {room['room_id']}: {room['issue']}")

🚨 Evacuation Route Analysis:
📊 Evacuation Statistics:
   Total Rooms: 9
   Average Distance: 0.0 m
   Compliance Rate: 0.0%

📋 Compliance Validation:
   Overall Status: NON_COMPLIANT
   Compliant Rooms: 0/9
   Non-Compliant: 0
   Inaccessible: 9

🚫 Inaccessible Rooms (no evacuation path):
   🏠 R_CSZ_34.0_(-0.50)_DEFAULT: No evacuation path available
   🏠 R_MUELLE_DEFAULT: No evacuation path available
   🏠 R_H_base_de_taller_DEFAULT: No evacuation path available
   🏠 R_PB_DEFAULT: No evacuation path available
   🏠 R_REF_CubiertaAnexoFrio_DEFAULT: No evacuation path available
   🏠 R_Altillo_DEFAULT: No evacuation path available
   🏠 R_PANEL_PREFABRICADO_DEFAULT: No evacuation path available
   🏠 R_Ref_PetoAnexoFrio_DEFAULT: No evacuation path available
   🏠 R_Ref_H_Peto_Max_DEFAULT: No evacuation path available


## 🚪 Step 4: Door Usage and Compliance Analysis

Let's analyze how doors are used and their compliance:

In [ ]:
# Door usage and compliance analysis with VISUALIZATION
from src.calculations.graph import calculate_door_usage_analysis

print("🚪 Door Usage & Compliance Analysis:")
print("=" * 60)

# Analyze door usage in evacuation routes
door_usage = calculate_door_usage_analysis(project)

print(f"📊 Door Usage Statistics:")
print(f"   Total Usage Count: {door_usage['total_usage']}")
print(f"   High Usage Doors: {door_usage['usage_distribution']['high_usage']}")
print(f"   Medium Usage Doors: {door_usage['usage_distribution']['medium_usage']}")
print(f"   Unused Doors: {door_usage['usage_distribution']['unused']}")

# 🎨 VISUALIZATION: Door Analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Left: Door width compliance visualization
door_widths = [door['width_mm'] for door in loader.all_doors]
door_types = [door.get('door_type', 'unknown') for door in loader.all_doors]
emergency_exits = [door.get('is_emergency_exit', False) for door in loader.all_doors]

# Create scatter plot
colors = ['red' if is_exit else 'blue' for is_exit in emergency_exits]
sizes = [100 if is_exit else 50 for is_exit in emergency_exits]

scatter = ax1.scatter(range(len(door_widths)), door_widths, 
                     c=colors, s=sizes, alpha=0.7, edgecolors='black')

# Add compliance lines
ax1.axhline(y=800, color='orange', linestyle='--', alpha=0.7, label='Min Regular (800mm)')
ax1.axhline(y=900, color='red', linestyle='--', alpha=0.7, label='Min Emergency (900mm)')

ax1.set_xlabel('Door Index', fontweight='bold')
ax1.set_ylabel('Door Width (mm)', fontweight='bold')
ax1.set_title('Door Width Compliance Analysis', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Add compliance status text
compliant_doors = sum(1 for door in loader.all_doors 
                     if door['width_mm'] >= (900 if door.get('is_emergency_exit', False) else 800))
compliance_rate = compliant_doors / len(loader.all_doors) if loader.all_doors else 0

ax1.text(0.02, 0.98, f'Compliance Rate: {compliance_rate:.1%}\nRed = Emergency Exit\nBlue = Regular Door', 
        transform=ax1.transAxes, va='top', ha='left',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Right: Door distribution by level
level_door_counts = {}
for level in loader.levels:
    level_doors = level.get('doors', [])
    level_door_counts[level['name']] = len(level_doors)

if level_door_counts:
    levels = list(level_door_counts.keys())
    counts = list(level_door_counts.values())
    
    # Create pie chart
    colors_pie = plt.cm.Set3(range(len(levels)))
    wedges, texts, autotexts = ax2.pie(counts, labels=[l.split('_')[0] if '_' in l else l[:10] for l in levels], 
                                      autopct='%1.0f', colors=colors_pie, startangle=90)
    
    ax2.set_title('Door Distribution by Level', fontsize=14, fontweight='bold')
    
    # Improve text readability
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')

plt.tight_layout()
plt.show()

# Door compliance analysis (simplified)
emergency_exits = sum(1 for d in loader.all_doors if d.get('is_emergency_exit', False))
print(f"\n📏 Door Compliance Summary:")
print(f"   Total Doors: {len(loader.all_doors)}")
print(f"   Emergency Exits: {emergency_exits}")
print(f"   Compliance Rate: {compliance_rate:.1%}")

if compliance_rate >= 0.95:
    print(f"   Status: ✅ Excellent compliance")
elif compliance_rate >= 0.80:
    print(f"   Status: ⚠️ Good compliance")
else:
    print(f"   Status: ❌ Poor compliance")

print(f"\n🎨 Visualization shows:")
print(f"   📊 Left: Door widths vs compliance thresholds")
print(f"   🥧 Right: Distribution of doors across building levels")

## 👥 Step 5: Occupancy Load Analysis

Let's calculate occupancy loads and egress capacity:

In [ ]:
# Occupancy load analysis with VISUALIZATION

print("👥 Occupancy Load Analysis:")
print("=" * 60)

# Analyze occupancy for each room
total_occupancy = 0
room_analysis = []

for room in loader.all_rooms:
    area = room['area']
    use_type = room.get('use', 'commercial')
    
    # Calculate occupancy based on use
    occupancy_factors = {
        "industrial": 0.02,    # 1 person per 50 sqm
        "warehouse": 0.02,    # 1 person per 50 sqm
        "office": 0.1,        # 1 person per 10 sqm
        "commercial": 0.1,    # 1 person per 10 sqm
        "assembly": 0.5,      # 1 person per 2 sqm
        "storage": 0.02,      # 1 person per 50 sqm
    }
    
    factor = occupancy_factors.get(use_type, 0.05)  # Default
    occupancy = max(1, int(area * factor))
    
    # Required egress width (5mm per person)
    required_width = occupancy * 5
    
    room_analysis.append({
        'id': room['id'],
        'name': room.get('name', room['id']),
        'area': area,
        'use': use_type,
        'occupancy': occupancy,
        'required_width': required_width,
        'density': occupancy / area
    })
    
    total_occupancy += occupancy

# Sort by occupancy (highest first)
room_analysis.sort(key=lambda x: x['occupancy'], reverse=True)

print(f"📊 Building Occupancy Summary:")
print(f"   Total Building Occupancy: {total_occupancy} people")
print(f"   Average per Room: {total_occupancy / len(room_analysis):.1f} people")
print(f"   Total Required Egress Width: {total_occupancy * 5:.0f} mm")

# 🎨 VISUALIZATION: Occupancy Analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Left: Room occupancy bar chart
room_names = [r['id'].split('_')[1] if '_' in r['id'] else r['id'][:8] for r in room_analysis]
occupancies = [r['occupancy'] for r in room_analysis]
areas = [r['area'] for r in room_analysis]

# Create bar chart with color based on density
densities = [r['density'] for r in room_analysis]
max_density = max(densities) if densities else 1
colors = plt.cm.RdYlBu_r([d/max_density for d in densities])

bars = ax1.bar(range(len(room_names)), occupancies, color=colors, alpha=0.8, edgecolor='black')

ax1.set_xticks(range(len(room_names)))
ax1.set_xticklabels(room_names, rotation=45, ha='right')
ax1.set_ylabel('Occupancy (people)', fontweight='bold')
ax1.set_title('Room Occupancy Levels', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, (bar, value) in enumerate(zip(bars, occupancies)):
    ax1.text(i, value + 0.1, str(value), ha='center', va='bottom', fontweight='bold')

# Add colorbar for density
sm = plt.cm.ScalarMappable(cmap=plt.cm.RdYlBu_r, norm=plt.Normalize(vmin=0, vmax=max_density))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax1, shrink=0.8)
cbar.set_label('Occupancy Density (people/m²)', fontweight='bold')

# Right: Area vs Occupancy scatter plot
ax2.scatter(areas, occupancies, c=densities, cmap='RdYlBu_r', 
           s=100, alpha=0.7, edgecolors='black')

ax2.set_xlabel('Room Area (m²)', fontweight='bold')
ax2.set_ylabel('Occupancy (people)', fontweight='bold')
ax2.set_title('Area vs Occupancy Relationship', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Add trend line
if len(areas) > 1:
    z = np.polyfit(areas, occupancies, 1)
    p = np.poly1d(z)
    ax2.plot(areas, p(areas), "r--", alpha=0.8, label=f'Trend: y={z[0]:.3f}x+{z[1]:.1f}')
    ax2.legend()

# Add density info
ax2.text(0.02, 0.98, f'Color = density\nTotal: {total_occupancy} people\nAvg density: {sum(densities)/len(densities):.3f} p/m²', 
        transform=ax2.transAxes, va='top', ha='left',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

# Find rooms with highest occupancy density
high_density_rooms = [r for r in room_analysis if r['density'] > 0.08]  # Threshold for demo

print(f"\n📈 Occupancy Analysis Results:")
if high_density_rooms:
    print(f"   ⚠️ High Density Rooms (>{0.08:.2f} people/m²):")
    for room in high_density_rooms[:3]:
        print(f"      🏠 {room['id']}: {room['density']:.3f} people/m² ({room['occupancy']} people)")
else:
    print(f"   ✅ All rooms have reasonable occupancy density")

print(f"\n🎨 Visualization shows:")
print(f"   📊 Left: Room occupancy levels (color = density)")
print(f"   📈 Right: Relationship between room area and occupancy")

## 📊 Step 6: Spatial Relationships Analysis

Let's analyze how rooms relate to each other spatially:

In [6]:
# Spatial relationships analysis
from src.calculations.graph import get_room_adjacency_list

print("📊 Spatial Relationships Analysis:")
print("=" * 60)

# Get room adjacency
adjacency = get_room_adjacency_list(project)

print(f"🔗 Room Connectivity Summary:")
print(f"   Total Rooms: {len(adjacency)}")

# Analyze connectivity patterns
connection_counts = [len(connections) for connections in adjacency.values()]
if connection_counts:
    avg_connections = sum(connection_counts) / len(connection_counts)
    max_connections = max(connection_counts)
    min_connections = min(connection_counts)
    
    print(f"   Average Connections per Room: {avg_connections:.1f}")
    print(f"   Range: {min_connections} - {max_connections} connections")

# Find hub rooms (highly connected)
hub_rooms = [(room_id, len(connections)) for room_id, connections in adjacency.items() 
             if len(connections) >= 3]
hub_rooms.sort(key=lambda x: x[1], reverse=True)

if hub_rooms:
    print(f"\n🎯 Hub Rooms (3+ connections):")
    for room_id, count in hub_rooms[:5]:
        # Find room name
        room_name = next((r.get('name', room_id) for r in loader.all_rooms if r['id'] == room_id), room_id)
        print(f"   🏠 {room_id}: {count} connections")
        
        # Show what it connects to
        connections = adjacency[room_id][:3]  # Show first 3
        if connections:
            print(f"      → Connected to: {', '.join(connections)}")
            if len(adjacency[room_id]) > 3:
                print(f"      → ... and {len(adjacency[room_id]) - 3} more")

# Find isolated rooms (no connections)
isolated_rooms = [room_id for room_id, connections in adjacency.items() if not connections]

if isolated_rooms:
    print(f"\n🏝️ Isolated Rooms (no connections):")
    for room_id in isolated_rooms[:5]:
        room_name = next((r.get('name', room_id) for r in loader.all_rooms if r['id'] == room_id), room_id)
        print(f"   🏠 {room_id}")
else:
    print(f"\n✅ All rooms are connected to the circulation network")

# Calculate building connectivity metrics
total_connections = sum(len(connections) for connections in adjacency.values())
connectivity_density = total_connections / (len(adjacency) * (len(adjacency) - 1)) if len(adjacency) > 1 else 0

print(f"\n📈 Building Connectivity Metrics:")
print(f"   Total Connections: {total_connections}")
print(f"   Connectivity Density: {connectivity_density:.3f}")
print(f"   Network Health: {'Good' if connectivity_density > 0.1 else 'Needs Improvement'}")

📊 Spatial Relationships Analysis:
🔗 Room Connectivity Summary:
   Total Rooms: 9
   Average Connections per Room: 0.2
   Range: 0 - 1 connections

🏝️ Isolated Rooms (no connections):
   🏠 R_CSZ_34.0_(-0.50)_DEFAULT
   🏠 R_H_base_de_taller_DEFAULT
   🏠 R_PB_DEFAULT
   🏠 R_Altillo_DEFAULT
   🏠 R_PANEL_PREFABRICADO_DEFAULT

📈 Building Connectivity Metrics:
   Total Connections: 2
   Connectivity Density: 0.028
   Network Health: Needs Improvement


## 🎯 Step 7: Room-Specific Analysis Tool

Let's create a tool to analyze any specific room in detail:

In [7]:
# Room-specific detailed analysis
from src.calculations.graph import calculate_room_connectivity_score

def analyze_specific_room(room_id: str):
    """Comprehensive analysis of a specific room."""
    
    print(f"🏠 Detailed Analysis: Room {room_id}")
    print("=" * 50)
    
    # Find room data
    room_data = next((r for r in loader.all_rooms if r['id'] == room_id), None)
    if not room_data:
        print(f"❌ Room {room_id} not found!")
        return
    
    # Basic room info
    print(f"📋 Basic Information:")
    print(f"   Name: {room_data.get('name', 'Unnamed')}")
    print(f"   Use: {room_data.get('use', 'Unknown')}")
    print(f"   Area: {room_data['area']:.1f} m²")
    print(f"   Level: {room_data.get('level', 'Unknown')}")
    
    # Connectivity analysis
    try:
        connectivity = calculate_room_connectivity_score(project, room_id)
        if 'error' not in connectivity:
            print(f"\n🔗 Connectivity Analysis:")
            print(f"   Direct Connections: {connectivity['direct_connections']}")
            print(f"   Reachable Rooms: {connectivity['reachable_rooms']}/{connectivity['total_rooms']}")
            print(f"   Reachability Score: {connectivity['reachability_score']:.2f}")
            print(f"   Average Distance to Others: {connectivity['average_distance']:.1f} m")
            print(f"   Connectivity Grade: {connectivity['connectivity_grade'].upper()}")
    except Exception as e:
        print(f"\n⚠️ Connectivity analysis unavailable: {str(e)}")
    
    # Evacuation analysis
    try:
        graph = create_circulation_graph(project)
        evacuation = graph.calculate_egress_distance(room_id)
        
        if 'error' not in evacuation:
            print(f"\n🚨 Evacuation Analysis:")
            print(f"   Distance to Exit: {evacuation['distance']:.1f} m")
            print(f"   Path: {' → '.join(evacuation['path'])}")
            print(f"   Accessible Route: {'✅ Yes' if evacuation['is_accessible'] else '❌ No'}")
            
            # Compliance check
            if evacuation['distance'] <= 25:
                print(f"   Compliance: ✅ Within 25m limit")
            elif evacuation['distance'] <= 35:
                print(f"   Compliance: ⚠️ Exceeds 25m but within 35m")
            else:
                print(f"   Compliance: ❌ Exceeds 35m limit")
    except Exception as e:
        print(f"\n⚠️ Evacuation analysis unavailable: {str(e)}")
    
    # Occupancy analysis
    area = room_data['area']
    use_type = room_data.get('use', 'commercial')
    
    occupancy_factors = {
        "industrial": 0.02, "warehouse": 0.02, "office": 0.1,
        "commercial": 0.1, "assembly": 0.5, "storage": 0.02
    }
    
    factor = occupancy_factors.get(use_type, 0.05)
    occupancy = max(1, int(area * factor))
    required_egress = occupancy * 5  # 5mm per person
    
    print(f"\n👥 Occupancy Analysis:")
    print(f"   Calculated Occupancy: {occupancy} people")
    print(f"   Occupancy Factor: {factor} people/m²")
    print(f"   Required Egress Width: {required_egress} mm")
    print(f"   Density: {occupancy/area:.3f} people/m²")

# Example: Analyze a specific room
print("🔍 Room Analysis Tool Demo:")
print("\nAvailable rooms:", [r['id'] for r in loader.all_rooms[:5]], "...")

# Analyze the first room as an example
if loader.all_rooms:
    example_room = loader.all_rooms[0]['id']
    print(f"\n📝 Example analysis for room: {example_room}")
    analyze_specific_room(example_room)
    
    print(f"\n💡 To analyze any room, call: analyze_specific_room('ROOM_ID')")

🔍 Room Analysis Tool Demo:

Available rooms: ['R_CSZ_34.0_(-0.50)_DEFAULT', 'R_MUELLE_DEFAULT', 'R_H_base_de_taller_DEFAULT', 'R_PB_DEFAULT', 'R_REF_CubiertaAnexoFrio_DEFAULT'] ...

📝 Example analysis for room: R_CSZ_34.0_(-0.50)_DEFAULT
🏠 Detailed Analysis: Room R_CSZ_34.0_(-0.50)_DEFAULT
📋 Basic Information:
   Name: General Space - CSZ 34.0 (-0.50)
   Use: commercial
   Area: 80.0 m²
   Level: CSZ 34.0 (-0.50)

🔗 Connectivity Analysis:
   Direct Connections: 0
   Reachable Rooms: 0/9
   Reachability Score: 0.00
   Average Distance to Others: 0.0 m
   Connectivity Grade: LOW

👥 Occupancy Analysis:
   Calculated Occupancy: 8 people
   Occupancy Factor: 0.1 people/m²
   Required Egress Width: 40 mm
   Density: 0.100 people/m²

💡 To analyze any room, call: analyze_specific_room('ROOM_ID')


## 📈 Step 8: Performance Summary Dashboard

Let's create a comprehensive building performance summary:

In [ ]:
# Building performance dashboard with VISUALIZATION
from src.calculations.graph import validate_evacuation_compliance, find_critical_circulation_points

print("📈 BUILDING PERFORMANCE DASHBOARD")
print("=" * 70)
print(f"Project: {loader.metadata.get('project_name')}")
print(f"Analysis Date: {loader.metadata.get('created_date', '')[:10]}")
print("=" * 70)

# Collect performance metrics
metrics = {}

# Overall metrics
metrics['total_area'] = loader.metadata.get('total_area', 0)
metrics['levels'] = len(loader.levels)
metrics['rooms'] = len(loader.all_rooms)
metrics['doors'] = len(loader.all_doors)
metrics['walls'] = len(loader.all_walls)

# Safety metrics
try:
    compliance = validate_evacuation_compliance(project)
    metrics['safety_compliance'] = compliance['compliance_rate']
    metrics['inaccessible_rooms'] = len(compliance['inaccessible_rooms'])
    safety_grade = "A" if compliance['compliance_rate'] >= 0.95 else "B" if compliance['compliance_rate'] >= 0.85 else "C" if compliance['compliance_rate'] >= 0.75 else "D"
    metrics['safety_grade'] = safety_grade
except Exception:
    metrics['safety_compliance'] = 0.0
    metrics['safety_grade'] = "D"

# Circulation metrics
try:
    critical_points = find_critical_circulation_points(project)
    metrics['critical_rooms'] = critical_points['critical_room_count']
    circulation_grade = "A" if critical_points['connectivity_risk'] == 'low' else "B" if critical_points['connectivity_risk'] == 'medium' else "C"
    metrics['circulation_grade'] = circulation_grade
except Exception:
    metrics['critical_rooms'] = 0
    metrics['circulation_grade'] = "C"

# Door compliance
emergency_exits = sum(1 for d in loader.all_doors if d.get('is_emergency_exit', False))
compliant_doors = sum(1 for d in loader.all_doors 
                     if d['width_mm'] >= (900 if d.get('is_emergency_exit', False) else 800))
door_compliance_rate = compliant_doors / len(loader.all_doors) if loader.all_doors else 0
door_grade = "A" if door_compliance_rate >= 0.95 else "B" if door_compliance_rate >= 0.85 else "C" if door_compliance_rate >= 0.75 else "D"

metrics['door_compliance'] = door_compliance_rate
metrics['door_grade'] = door_grade
metrics['emergency_exits'] = emergency_exits

# Occupancy metrics
total_occupancy = sum(max(1, int(r['area'] * 0.1)) for r in loader.all_rooms)  # Commercial factor
avg_density = total_occupancy / metrics['total_area']
metrics['total_occupancy'] = total_occupancy
metrics['avg_density'] = avg_density

# 🎨 VISUALIZATION: Performance Dashboard
fig = plt.figure(figsize=(18, 12))

# Create a 3x3 grid for different visualizations
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Overall building metrics (top left)
ax1 = fig.add_subplot(gs[0, 0])
building_metrics = ['Levels', 'Rooms', 'Doors', 'Walls']
building_values = [metrics['levels'], metrics['rooms'], metrics['doors'], metrics['walls']]
colors1 = plt.cm.Set2(range(len(building_metrics)))

bars1 = ax1.bar(building_metrics, building_values, color=colors1, alpha=0.8, edgecolor='black')
ax1.set_title('Building Overview', fontweight='bold', fontsize=12)
ax1.set_ylabel('Count')

for bar, value in zip(bars1, building_values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
            str(value), ha='center', va='bottom', fontweight='bold')

# 2. Compliance grades (top middle)
ax2 = fig.add_subplot(gs[0, 1])
grades = ['Safety', 'Circulation', 'Doors']
grade_values = [metrics['safety_grade'], metrics['circulation_grade'], metrics['door_grade']]
grade_scores = [{"A": 4, "B": 3, "C": 2, "D": 1}[g] for g in grade_values]
colors2 = ['green' if s >= 3 else 'orange' if s >= 2 else 'red' for s in grade_scores]

bars2 = ax2.bar(grades, grade_scores, color=colors2, alpha=0.8, edgecolor='black')
ax2.set_title('Compliance Grades', fontweight='bold', fontsize=12)
ax2.set_ylabel('Grade Score')
ax2.set_ylim(0, 4.5)
ax2.set_yticks([1, 2, 3, 4])
ax2.set_yticklabels(['D', 'C', 'B', 'A'])

for bar, grade in zip(bars2, grade_values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
            grade, ha='center', va='bottom', fontweight='bold', fontsize=14)

# 3. Occupancy analysis (top right)
ax3 = fig.add_subplot(gs[0, 2])
occupancy_data = ['Total Occupancy', 'Emergency Exits', 'Critical Rooms']
occupancy_values = [metrics['total_occupancy'], metrics['emergency_exits'], metrics['critical_rooms']]
colors3 = ['blue', 'red', 'orange']

bars3 = ax3.bar(occupancy_data, occupancy_values, color=colors3, alpha=0.8, edgecolor='black')
ax3.set_title('Safety Metrics', fontweight='bold', fontsize=12)
ax3.set_ylabel('Count')

for bar, value in zip(bars3, occupancy_values):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
            str(value), ha='center', va='bottom', fontweight='bold')

# 4. Compliance rates (middle left)
ax4 = fig.add_subplot(gs[1, 0])
compliance_labels = ['Safety\\nCompliance', 'Door\\nCompliance']
compliance_values = [metrics['safety_compliance'], metrics['door_compliance']]
colors4 = ['green' if v >= 0.8 else 'orange' if v >= 0.6 else 'red' for v in compliance_values]

bars4 = ax4.bar(compliance_labels, compliance_values, color=colors4, alpha=0.8, edgecolor='black')
ax4.set_title('Compliance Rates', fontweight='bold', fontsize=12)
ax4.set_ylabel('Compliance Rate')
ax4.set_ylim(0, 1.1)

for bar, value in zip(bars4, compliance_values):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{value:.1%}', ha='center', va='bottom', fontweight='bold')

# 5. Overall score gauge (middle center)
ax5 = fig.add_subplot(gs[1, 1])
# Calculate overall score
scores = []
if metrics['safety_grade']:
    safety_score = {"A": 95, "B": 85, "C": 75, "D": 65}[metrics['safety_grade']]
    scores.append(safety_score)
if metrics['circulation_grade']:
    circulation_score = {"A": 95, "B": 85, "C": 75}[metrics['circulation_grade']]
    scores.append(circulation_score)
door_score = {"A": 95, "B": 85, "C": 75, "D": 65}[metrics['door_grade']]
scores.append(door_score)

overall_score = sum(scores) / len(scores) if scores else 0

# Create a simple gauge
theta = np.linspace(0, np.pi, 100)
r = 1
x = r * np.cos(theta)
y = r * np.sin(theta)

ax5.plot(x, y, 'k-', linewidth=3)
ax5.fill_between(x, 0, y, alpha=0.3, color='lightgray')

# Color the gauge based on score
score_angle = (overall_score / 100) * np.pi
score_x = np.cos(score_angle)
score_y = np.sin(score_angle)

color = 'green' if overall_score >= 80 else 'orange' if overall_score >= 60 else 'red'
ax5.arrow(0, 0, score_x*0.8, score_y*0.8, head_width=0.1, head_length=0.1, 
         fc=color, ec=color, linewidth=3)

ax5.set_xlim(-1.2, 1.2)
ax5.set_ylim(-0.2, 1.2)
ax5.set_aspect('equal')
ax5.set_title(f'Overall Score: {overall_score:.0f}/100', fontweight='bold', fontsize=12)
ax5.text(0, -0.1, f'Rating: {"EXCELLENT" if overall_score >= 90 else "GOOD" if overall_score >= 80 else "ACCEPTABLE" if overall_score >= 70 else "NEEDS IMPROVEMENT"}', 
         ha='center', va='top', fontweight='bold')
ax5.axis('off')

# 6. Area distribution (middle right)
ax6 = fig.add_subplot(gs[1, 2])
area_per_room = [room['area'] for room in loader.all_rooms]
ax6.hist(area_per_room, bins=5, color='skyblue', alpha=0.8, edgecolor='black')
ax6.set_title('Room Area Distribution', fontweight='bold', fontsize=12)
ax6.set_xlabel('Room Area (m²)')
ax6.set_ylabel('Number of Rooms')

# 7. Performance summary table (bottom row)
ax7 = fig.add_subplot(gs[2, :])
ax7.axis('off')

# Create summary table
summary_data = [
    ['Metric', 'Value', 'Status'],
    ['Total Area', f"{metrics['total_area']:.0f} m²", '✅'],
    ['Safety Compliance', f"{metrics['safety_compliance']:.1%}", '✅' if metrics['safety_compliance'] > 0.8 else '❌'],
    ['Door Compliance', f"{metrics['door_compliance']:.1%}", '✅' if metrics['door_compliance'] > 0.95 else '❌'],
    ['Emergency Exits', f"{metrics['emergency_exits']}", '⚠️' if metrics['emergency_exits'] == 0 else '✅'],
    ['Critical Rooms', f"{metrics['critical_rooms']}", '✅' if metrics['critical_rooms'] == 0 else '⚠️'],
    ['Occupancy Density', f"{metrics['avg_density']:.3f} p/m²", '✅'],
    ['Overall Grade', f"{overall_score:.0f}/100", '✅' if overall_score >= 80 else '⚠️' if overall_score >= 60 else '❌']
]

table = ax7.table(cellText=summary_data[1:], colLabels=summary_data[0], 
                 cellLoc='center', loc='center', colWidths=[0.3, 0.3, 0.1])
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1.2, 2)

# Style the table
for i in range(len(summary_data)):
    for j in range(len(summary_data[0])):
        if i == 0:  # Header
            table[(i, j)].set_facecolor('#40466e')
            table[(i, j)].set_text_props(weight='bold', color='white')
        else:
            if j == 2:  # Status column
                if summary_data[i][j] == '✅':
                    table[(i, j)].set_facecolor('#90EE90')
                elif summary_data[i][j] == '❌':
                    table[(i, j)].set_facecolor('#FFB6C1')
                else:
                    table[(i, j)].set_facecolor('#FFE4B5')

plt.suptitle('🏗️ Building Performance Dashboard - Vilamalla Industrial Complex', fontsize=16, fontweight='bold', y=0.98)
plt.show()

print(f"\n🎨 Performance Dashboard shows:")
print(f"   📊 Building overview and key metrics")
print(f"   📈 Compliance grades and performance scores")
print(f"   🎯 Overall building rating: {overall_score:.0f}/100")
print(f"   📋 Comprehensive summary table")

## 🎯 Summary: Visual Computational Tools Tutorial

This notebook demonstrated the **practical computational tools** our compliance agent uses, with **clear visualizations** to show what each tool does:

### 🎨 **Visual Analysis Tools Demonstrated**:

1. **🔗 Building Connectivity Analysis** 
   - **Left Chart**: Network graph showing room connections (red = critical)
   - **Right Chart**: Connectivity levels by room (green = well connected)
   - **What it shows**: How rooms connect and which are critical for circulation

2. **🚪 Door Usage & Compliance Analysis**
   - **Left Chart**: Door widths vs compliance thresholds (scatter plot)
   - **Right Chart**: Door distribution across building levels (pie chart)
   - **What it shows**: Which doors meet regulations and how they're distributed

3. **👥 Occupancy Load Analysis**
   - **Left Chart**: Room occupancy levels with density color coding
   - **Right Chart**: Area vs occupancy relationship with trend line
   - **What it shows**: How many people each room can hold and density patterns

4. **📈 Performance Dashboard**
   - **Comprehensive Grid**: 7 different visualization panels
   - **Gauge Chart**: Overall building performance score
   - **Summary Table**: Complete metrics with color-coded status
   - **What it shows**: Complete building health at a glance

### 🔧 **Key Visual Benefits**:
- **Immediate Understanding**: See problems and patterns instantly
- **Color-Coded Results**: Green = good, orange = warning, red = issue
- **Multiple Perspectives**: Different chart types show different aspects
- **Professional Presentation**: Ready for reports and presentations

### 🎪 **For Tutorial Users**:
- **Visual Learning**: Understand what tools do by seeing results
- **Pattern Recognition**: Spot issues across the building quickly
- **Comparative Analysis**: Compare rooms, doors, and levels visually
- **Decision Support**: Clear visual data for informed decisions

### 🚀 **For the Agent**:
These same computational tools work behind the scenes to:
- **Analyze Building Performance**: All calculations shown here
- **Generate Visual Reports**: Create these charts automatically
- **Identify Issues**: Flag problems the visualizations reveal
- **Support Decisions**: Provide data for compliance recommendations

### 💡 **What You Learned**:
- How to **visualize circulation patterns** in buildings
- How to **identify compliance issues** through charts
- How to **analyze occupancy distribution** visually
- How to create **comprehensive dashboards** for building performance

**The visualizations make complex building analysis accessible and actionable!** 📊✨